In [2]:
TF_ENABLE_ONEDNN_OPTS=0

In [3]:
import os 
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm.auto import tqdm
import cv2

# from tensorflow.keras.applications import Densnet
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
import tensorflow.keras as keras
from tensorflow.data import Dataset

from tensorflow.keras.optimizers import Adam, RMSprop, Nadam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from itertools import product
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications import *
from sklearn.preprocessing import LabelEncoder

/home/ubuntu/anaconda3/envs/smartpot/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
BATCH_SIZE = 20
IMAGE_SIZE = 320

In [5]:
root_dir = '/home/lab38/images'
sample_csv = '딸기_정상_수정본.csv'
df = pd.read_csv(os.path.join(root_dir, sample_csv))
df.head()

,image,date,height,width,type,disease,crop,area,grow,risk,points
0,V006_77_0_00_04_01_13_0_b02_20201203_0000_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1167, 'xbr': 2255, 'ybr'..."
1,V006_77_0_00_04_01_13_0_b02_20201203_0001_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 600, 'ytl': 1087, 'xbr': 1628, 'ybr':..."
2,V006_77_0_00_04_01_13_0_b02_20201203_0002_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1217, 'xbr': 2503, 'ybr'..."
3,V006_77_0_00_04_01_13_0_b02_20201203_0003_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 662, 'ytl': 1252, 'xbr': 2129, 'ybr':..."
4,V006_77_0_00_04_01_13_0_b02_20201203_0004_S01_...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 370, 'ytl': 884, 'xbr': 1994, 'ybr': ..."


In [22]:
df_s = df.copy()
df_s.drop(['date', 'height', 'width', 'type', 'risk'], axis=1, inplace=True)
df_s.head()

,image,disease,crop,area,grow,points
0,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,0,4,0,1,"[{'xtl': 1003, 'ytl': 1167, 'xbr': 2255, 'ybr'..."
1,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,0,4,0,1,"[{'xtl': 600, 'ytl': 1087, 'xbr': 1628, 'ybr':..."
2,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,0,4,0,1,"[{'xtl': 1003, 'ytl': 1217, 'xbr': 2503, 'ybr'..."
3,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,0,4,0,1,"[{'xtl': 662, 'ytl': 1252, 'xbr': 2129, 'ybr':..."
4,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,0,4,0,1,"[{'xtl': 370, 'ytl': 884, 'xbr': 1994, 'ybr': ..."


In [20]:
glob('/home/lab38/images/*.csv')

['/home/lab38/images/딸기_정상_수정본.csv']

In [6]:
sample_img_path = os.path.join(root_dir, '딸기_정상')
df['image'] = df['image'].map(lambda x : os.path.join(sample_img_path, x))

In [7]:
df.head()

,image,date,height,width,type,disease,crop,area,grow,risk,points
0,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1167, 'xbr': 2255, 'ybr'..."
1,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 600, 'ytl': 1087, 'xbr': 1628, 'ybr':..."
2,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 1003, 'ytl': 1217, 'xbr': 2503, 'ybr'..."
3,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 662, 'ytl': 1252, 'xbr': 2129, 'ybr':..."
4,/home/lab38/images/딸기_정상/V006_77_0_00_04_01_13...,2020-12-03,4032,3024,0,0,4,1,13,0,"[{'xtl': 370, 'ytl': 884, 'xbr': 1994, 'ybr': ..."


Scikit learn Label Encodor 

In [8]:
a_en = LabelEncoder()
g_en = LabelEncoder()
d_en = LabelEncoder()

a_en.fit(df['area'])
g_en.fit(df['grow'])
d_en.fit(df['disease'])

df['area'] = a_en.transform(df['area'])
df['grow'] = g_en.transform(df['grow'])
df['disease'] = d_en.transform(df['disease'])

In [9]:
shuffled_df = df.sample(frac=1)
# Train : Validation = 0.8 : 0.2 Split
train_len = int(len(df)*0.8)

train_df = shuffled_df[:100]
valid_df = shuffled_df[-100:]

train_img_path = train_df['image'].to_list()
train_label = train_df['grow'].to_list()

valid_img_path = valid_df['image'].to_list()
valid_label = valid_df['grow'].to_list()

### Try 1 : Custom Data Generator

In [121]:
class CustomDataGen(tf.keras.utils.Sequence):
    def __init__(self, df, X_col, y_col,
                 batch_size,
                 input_size=(224, 224, 3),
                 shuffle=True):
        
        self.df = df.copy()
        self.X_col = X_col
        self.y_col = y_col
        self.batch_size = batch_size
        self.input_size = input_size
        self.shuffle = shuffle
        
        self.n = len(self.df)
        
        self.n_cls1 = df[self.y_col['cls1']].nunique()
#         self.cls1_en = LabelEncoder()
#         self.cls1_en.fit(self.df[self.y_col['cls1']])
#         self.df[self.y_col['cls1']] = self.cls1_en.transform(self.df[self.y_col['cls1']])
        
        self.n_cls2 = df[self.y_col['cls2']].nunique()
#         self.cls2_en = LabelEncoder()
#         self.cls2_en.fit(self.df[self.y_col['cls2']])
#         self.df[self.y_col['cls2']] = self.cls2_en.transform(self.df[self.y_col['cls2']])
        
        self.n_cls3 = df[self.y_col['cls3']].nunique()
#         self.cls3_en = LabelEncoder()
#         self.cls3_en.fit(self.df[self.y_col['cls3']])
#         self.df[self.y_col['cls3']] = self.cls3_en.transform(self.df[self.y_col['cls3']])
#         self.n_type = df[y_col['type']].nunique()
        print(self.n_cls1, self.n_cls2, self.n_cls3)
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)
    
    def __get_input(self, path, target_size):
    
#         xmin, ymin, w, h = bbox['x'], bbox['y'], bbox['width'], bbox['height']

        image = tf.keras.preprocessing.image.load_img(path)
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = image / 255.
        image = tf.image.resize(image, (target_size[0], target_size[1])).numpy()
        
#         image_arr = image_arr[ymin:ymin+h, xmin:xmin+w]
#         image_arr = tf.image.resize(image_arr,(target_size[0], target_size[1])).numpy() 
        
        return image
    
    def __get_output(self, label, num_classes):
        return label
#         return tf.keras.utils.to_categorical(label, num_classes=num_classes)

    def __get_data(self, batches):
        # Generates data containing batch_size samples
        path_batch = batches[self.X_col['path']]
#         bbox_batch = batches[self.X_col['bbox']]
        
        cls1_batch = batches[self.y_col['cls1']]
        cls2_batch = batches[self.y_col['cls2']]
        cls3_batch = batches[self.y_col['cls3']]
#         type_batch = batches[self.y_col['type']]

        X_batch = np.asarray([self.__get_input(x, self.input_size) for x in path_batch])

        y1_batch = np.asarray([self.__get_output(y, self.n_cls1) for y in cls1_batch])
        y2_batch = np.asarray([self.__get_output(y, self.n_cls2) for y in cls2_batch])
        y3_batch = np.asarray([self.__get_output(y, self.n_cls3) for y in cls3_batch])
#         y1_batch = np.asarray([self.__get_output(y, self.n_type) for y in type_batch])

        print(y1_batch.shape, y1_batch.dtype, type(y1_batch))
        print(pd.DataFrame(y1_batch))
        print(y2_batch.shape)
        print(pd.DataFrame(y2_batch))
        print(y3_batch.shape)
        print(pd.DataFrame(y2_batch))
        print(np.asarray([y1_batch, y2_batch, y3_batch]).shape)
        #return X_batch, np.asarray([y1_batch, y2_batch, y3_batch])
        return X_batch, y1_batch, y2_batch, y3_batch
    
    def __getitem__(self, index):
        
        batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y1, y2, y3 = self.__get_data(batches)        
        yield X, {'a':y1, 'g':y2, 'd':y3}
    
    def __len__(self):
        return self.n // self.batch_size


In [122]:
traingen = CustomDataGen(train_df,
                        X_col = {'path':'image'},
                        y_col = {'cls1' : 'area', 'cls2' : 'disease', 'cls3':'grow'},
                        batch_size = 2,
                        input_size = (IMAGE_SIZE, IMAGE_SIZE, 3))
validgen = CustomDataGen(valid_df,
                        X_col = {'path':'image'},
                        y_col = {'cls1' : 'area', 'cls2' : 'disease', 'cls3':'grow'},
                        batch_size = 2,
                        input_size = (IMAGE_SIZE, IMAGE_SIZE, 3))

4 1 2
4 1 2


## Try 2 : keras ImageDataGeneraotor, flow from dataframe

In [10]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255,
                            rotation_range=20, # 최대 20도까지 회전
                            width_shift_range=0.1, # 최대 x 범위안에서 좌우/상하 이동
                            height_shift_range=0.1,
                            zoom_range=0.2, # 확대 축소 비율,
                            horizontal_flip=True, # 좌우반전
                            vertical_flip=True, # 상하반전
                            fill_mode='nearest')
valid_gen = ImageDataGenerator(rescale= 1. /255.)

In [11]:
train_generator = train_gen.flow_from_dataframe(train_df, 
                                               x_col='image',
                                               y_col=['area', 'disease', 'grow'],
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='multi_output',
                                               batch_size=BATCH_SIZE)

Found 100 validated image filenames.


In [12]:
valid_generator = valid_gen.flow_from_dataframe(valid_df,
                                               x_col='image',
                                               y_col=['area','disease','grow'],
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               class_mode='multi_output',
                                               batch_size=BATCH_SIZE)

Found 100 validated image filenames.


Check

In [13]:
X, y = train_generator[0]

In [14]:
X.shape

(20, 320, 320, 3)

In [15]:
y

[array([3, 0, 1, 2, 1, 1, 1, 1, 1, 3, 2, 2, 1, 3, 2, 2, 0, 1, 1, 1]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1])]

---  
# Multi Output Model

In [17]:
inputs = keras.Input(shape=(320, 320, 3), name="img")
x = layers.Conv2D(16, 3, activation="relu")(inputs)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32, activation='relu')(x)
a_out = layers.Dense(df['area'].nunique(), name='a', activation='softmax')(x)
g_out = layers.Dense(df['grow'].nunique(), name='g', activation='softmax')(x)
d_out = layers.Dense(df['disease'].nunique(), name='d', activation='softmax')(x)
model = keras.Model(inputs, [a_out, d_out, g_out], name="test")
model.summary()

Model: "test"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 320, 320, 3) 0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 318, 318, 16) 448         img[0][0]                        
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 316, 316, 32) 4640        conv2d_2[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 105, 105, 32) 0           conv2d_3[0][0]                   
_______________________________________________________________________________________________

딸기 정상 데이터 셋에는 diease 클래스 값이 0(정상) 하나만 존재해서  
딸기 질병 데이터 셋과 합치고 모델 세부적인 값을 수정해야할 필요가 있음

In [18]:
model.compile(loss={ 'a' : 'sparse_categorical_crossentropy',
                     'g' : 'sparse_categorical_crossentropy',
                     'd' : 'sparse_categorical_crossentropy'},
             optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [19]:
model.fit(train_generator,
         validation_data=valid_generator,
         epochs=1,
         verbose=1,
         steps_per_epoch = 5)

2022-06-10 15:42:51.382240: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-10 15:42:51.399726: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2500000000 Hz


5/5 [==============================] - 39s 8s/step - loss: 2.1870 - a_loss: 1.4651 - d_loss: 0.0000e+00 - g_loss: 0.7219 - a_accuracy: 0.2925 - d_accuracy: 0.0000e+00 - g_accuracy: 0.5197 - val_loss: 1.8581 - val_a_loss: 1.2733 - val_d_loss: 0.0000e+00 - val_g_loss: 0.5848 - val_a_accuracy: 0.4700 - val_d_accuracy: 0.0000e+00 - val_g_accuracy: 0.7800


# 참고

https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3

https://stackoverflow.com/questions/38972380/keras-how-to-use-fit-generator-with-multiple-outputs-of-different-type

https://datascience.stackexchange.com/questions/74911/data-augmentation-for-multiple-output-heads-in-keras

https://medium.com/greendeck/keras-data-generator-for-multi-output-models-ea0408c76e8e